In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, SequentialSampler,DataLoader
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.preprocessing import MinMaxScaler

In [2]:
data_dir = 'data/'

In [3]:
class SlidingWindowDataset(Dataset):
    """Sliding window dataset"""
    
    def __init__(self,csv_data,timesteps,sliding_window):
        self.data = csv_data
        self.timesteps = timesteps
        self.sliding_window = sliding_window
    
    def __getitem__(self,index):
        x = self.data.iloc[:, index: index + self.timesteps]
        y = self.data.iloc[:, index + self.timesteps : index+self.sliding_window ]
        x = torch.tensor(x.values).type(dtype=torch.float)
        y = torch.tensor(y.values).type(dtype=torch.float)
        return x,y
    
    def __len__(self):
        return self.data.shape[1] - self.sliding_window + 1

In [4]:
# read data (for now, sell_prices & calendar are not used)

data_dir = 'data/'

train_sales = pd.read_csv(data_dir + 'sales_train_validation.csv')
#sell_prices = pd.read_csv(data_dir + 'sell_prices.csv')
#calendar = pd.read_csv(data_dir + 'calendar.csv')
submission_file = pd.read_csv(data_dir + 'sample_submission.csv')

In [5]:
# create training data, for now it only contains the sales and no extra features
sales = train_sales.drop(["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"], axis=1)

# normalize training data
scaler = MinMaxScaler()
scaler.fit(sales.T)
sales = pd.DataFrame(scaler.transform(sales.T))
sales = sales.T

sales.head()

,0,1,2,3,4,5,6,7,8,9,...,1903,1904,1905,1906,1907,1908,1909,1910,1911,1912
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.200000,0.600000,0.000000,0.200000,0.200000,0.200000,0.600000,0.000000,0.200000,0.200000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.333333,0.166667,0.333333,0.166667,0.166667,0.166667,0.000000,0.166667,0.166667,0.166667
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.066667,0.000000,0.333333,0.266667,0.066667,0.000000,0.066667,0.200000,0.466667,0.133333
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.222222,0.111111,0.111111,0.000000,0.111111,0.111111,0.222222,0.222222,0.222222,0.444444


In [6]:
timesteps = 14
prediction_steps = 28
len_window = timesteps + prediction_steps

In [7]:
dataset = SlidingWindowDataset(sales, timesteps, len_window)
dataset_sampler_loader = DataLoader(dataset, batch_size=32, shuffle=False)

In [8]:
class LSTM(nn.Module):
    def __init__(self,prediction_steps):
        super(LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size=14,hidden_size=10,num_layers=2,bidirectional=True)
        self.lin = nn.Linear(20,28)
        
    def forward(self,x):
        x, _ = self.lstm1(x)
        x = self.lin(x)
        return x

In [9]:
lstm = LSTM(prediction_steps)

In [10]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.001, weight_decay=1e-5)

In [11]:
for epoch in range(5):
        
    for batch_id, (sliding_window, label) in tqdm(enumerate(dataset_sampler_loader), total=len(dataset_sampler_loader), desc="#train batches", leave=False):
       
        lstm.train()
        outputs = lstm(sliding_window)
        optimizer.zero_grad()
        loss = criterion(outputs,label)

        loss.backward()
        optimizer.step()

    print("Epoch: %d, loss: %1.5f  " %(epoch, loss.cpu().item()))

Epoch: 0, loss: 0.02186  


Epoch: 1, loss: 0.01956  


Epoch: 2, loss: 0.01879  


Epoch: 3, loss: 0.01857  


Epoch: 4, loss: 0.01833  


In [50]:
lstm.eval()

# get input data for predictions
X_pred = sales.iloc[:,-timesteps:].to_numpy()
X_pred = X_pred.reshape(1, X_pred.shape[0], X_pred.shape[1])
X_pred = torch.tensor(X_pred).type(dtype=torch.float)

# get predictions
norm_pred = lstm(X_pred)[0].detach().numpy()
predictions = scaler.inverse_transform(norm_pred.T)
predictions = np.round(np.abs(predictions)).T

In [51]:
# create submission file

validation = pd.concat([pd.DataFrame(predictions[:,0:prediction_steps]), pd.DataFrame(predictions[:,-prediction_steps:])])
validation = validation.astype(int)

validation.reset_index(inplace=True, drop=True)

validation['id'] = submission_file.id
validation = validation.reindex(columns=['id'] + [c for c in validation.columns if c != 'id'], copy=False)

validation.columns = ['id'] + [f"F{i}" for i in range(1, 29)]

validation.to_csv('submission.csv', index=False)